# ElasticsearchVectorIndexer 사용 예시
이 노트북은 ElasticsearchVectorIndexer를 사용하여 Elasticsearch에 문장 임베딩을 색인하는 방법을 보여줍니다.

In [1]:
from vector_indexer import ElasticsearchVectorIndexer
import logging

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

### 1. ElasticsearchVectorIndexer 초기화
Elasticsearch와의 연결을 설정하고, SentenceTransformer 모델을 초기화합니다.

In [3]:
HOST = "localhost"
PORT = 9206
USERNAME = None
PASSWORD = None
SBERT_MODEL_NAME = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"

indexer = ElasticsearchVectorIndexer(host=HOST, port=PORT, username=USERNAME, password=PASSWORD, sbert_model_name=SBERT_MODEL_NAME)

2023-08-24 11:51:30,624 - INFO - Load pretrained SentenceTransformer: snunlp/KR-SBERT-V40K-klueNLI-augSTS
2023-08-24 11:51:35,352 - INFO - Use pytorch device: cuda
2023-08-24 11:51:35,354 - INFO - Initialized with SBERT model: snunlp/KR-SBERT-V40K-klueNLI-augSTS


### 2. Elasticsearch에서 데이터 가져오기
원본 데이터를 Elasticsearch에서 가져옵니다.

In [ ]:
SOURCE_INDEX = "원본_인덱스_이름"  # 사용자의 원본 인덱스 이름으로 변경
QUERY_SIZE = 1000  # Elasticsearch에서 가져올 문서의 수
EMBEDDING_TEXT_FIELD = ['text']  # 임베딩을 위한 데이터의 텍스트 필드 경로

data = indexer.get_data_from_es(SOURCE_INDEX, query_size=QUERY_SIZE, embedding_text_field=EMBEDDING_TEXT_FIELD)

### 3. 가져온 데이터에 대한 임베딩 생성
SentenceTransformer를 사용하여 문장의 임베딩을 생성합니다.

In [5]:
embeddings = indexer.embed_data(data, embedding_text_field=EMBEDDING_TEXT_FIELD)

2023-08-24 11:51:35,675 - INFO - Generating embeddings for 1000 sentences


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

2023-08-24 11:51:40,905 - INFO - Embeddings generated successfully


### 4. 생성된 임베딩을 Elasticsearch에 색인
생성된 임베딩을 Elasticsearch에 색인합니다. 원본 인덱스와 대상 인덱스가 동일한 경우 문서를 업데이트하며, 그렇지 않은 경우 새로운 문서를 색인합니다.

In [ ]:
TARGET_INDEX = "대상_인덱스_이름"  # 사용자의 대상 인덱스 이름으로 변경
DECIMAL_PRECISION = 5  # 임베딩 값의 소수점 자리 수

update = SOURCE_INDEX == TARGET_INDEX
indexer.index_data(TARGET_INDEX, data, embeddings, embedding_text_field=EMBEDDING_TEXT_FIELD, decimal_precision=DECIMAL_PRECISION, update=update)